# PawsPlace

Im Rahmen unseres Studienmoduls "Geomarketing mit Geodaten" entwickeln wir eine spezialisierte Anwendung, die darauf ausgerichtet ist, Hundehaltern bei der Wohnungssuche zu assistieren, indem sie Grünflächen in der Nähe einer eingegebenen Adresse identifiziert. Diese Anwendung verwendet Geodaten, um die Wohngebiete auf ihre Tauglichkeit für Hundebesitzer zu prüfen, speziell in Bezug auf die Verfügbarkeit und Erreichbarkeit von Grünflächen.


# Installationen

In folgendem Kapitel werden alle für das Projekt nötige Installationen vorgenommen. Falls Sie diese bereits installiert haben, können Sie die Zelle auslassen.

In [1]:
#!pip install osmnx
#!pip install geopandas
#!pip install folium
#!pip install pandas
# !pip install openrouteservice



Nun sind die ersten Installationen erledigt. Als nächstes wird OpenStreetMap installiert und vorbereitet.

In [2]:
import osmnx as ox
import geopandas as gpd
from IPython.display import display, clear_output
 
# Konfigurieren von OpenStreetMap
ox.config(log_console=True, use_cache=True)

C:\Users\j17jo\AppData\Local\Temp\ipykernel_28416\3790103234.py:6: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(log_console=True, use_cache=True)


## Vorbereitung API OpenRouteService

Um die Routen auf unserer Karte zu berechnen, wird OpenRouteService verwendet. Der API Key ist in der key.txt Datei hinterlegt. Folgende funktion extrahiert diesen und speichert den Key in einer Variable, damit die API genutzt werden kann.

In [3]:
def read_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.readline().strip()
    return api_key

file_path = 'key.txt'
api_key = read_api_key(file_path)

## Eingabe User

In diesem Kapitel wird die Adresse und die Angaben zum Spaziergang angegeben. Im späteren Verlauf des Notebooks wird noch ein UseCase eines Benutzers simuliert. Dieses Kapitel dient lediglich zur Demonstartion des Codes und der Ermittlung der Möglichkeiten

Zur Demonstration wurden folgende Angaben vorbereitet:

In [4]:
adresse = "Landstrasse 11, Turgi, Switzerland"
dauerSpaziergang = 20 # Minuten

## Berechnung Route / Kilometer des Spaziergangs

In [5]:
# Umrechnen der Spaziergangsdauer in Minuten in Distanz in Metern
# Annahme: Durchschnittliche Gehgeschwindigkeit = 100 Meter pro Minute
streckeSpaziergang = (dauerSpaziergang * 100)/2
print("Distanz in Metern: ", streckeSpaziergang)

Distanz in Metern:  1000.0


# Funktion zum Laden der Daten und Anzeigen auf einer Karte

Als erstes wird Anhand der Distanz in Meter, ermittelt welche Grünflächen sich in diesem Umkreis befinden. Die Distanz wurd in unserem Beispiel halbiert, da wir davon ausgehen, dass wir zu einer Grünfläche laufen und dann wieder zurück. Somit werden nur Grünflöcher ermittelt, die halb so weit entfernt sind.

Diese werden in der folgenden Karte angezeigt.

In [6]:
import osmnx as ox
import folium

def load_and_display(adresse, streckeSpaziergang):
    try:
        # Geocode Adresse
        location = ox.geocoder.geocode(adresse)
                
        # Suche nach Grünflächen in der Nähe
        tags = {'landuse': ['greenfield', 'grass','forest'],
        'leisure': 'park'}

        # Suche nach Grünflächen gemäss Userinput
        gdf = ox.features_from_point(location, tags, dist=streckeSpaziergang)
        
        # Karte erstellen
        m = folium.Map(location=[location[0], location[1]], zoom_start=14)
        folium.Marker([location[0], location[1]], popup='Eingegebene Adresse').add_to(m)
        
        # Grünflächen hinzufügen/markieren
        for _, row in gdf.iterrows():
            if row['geometry'].geom_type == 'Polygon':
                folium.Polygon(
                    locations=[(lat, lon) for lon, lat in row['geometry'].exterior.coords],
                    color='green', fill=True, fill_color='green'
                ).add_to(m)
        
        display(m)
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

# Aufruf der Funktion mit Benutzereingaben
load_and_display(adresse, streckeSpaziergang)


In einem nächsten Schritt wird OpenRouteService implementiert. Hiermit möchten wir ermitteln, welche Zonen in der vorgegebenen Zeit zu Fuss erreichbar sind. Diese Zone wird auf der Karte blau hinterlegt. 

### Isochronen & Grünflächen zusammen anzeigen

In [7]:
import osmnx as ox
import geopandas as gpd
import folium
from IPython.display import display
import openrouteservice
from openrouteservice import client as ors_client
from shapely.geometry import shape
from shapely.ops import unary_union

def create_isochrone(address, minutes, api_key):
    client = ors_client.Client(key=api_key)
    location = ox.geocode(address)
    coords = (location[1], location[0])

    params = {
        'locations': [coords],
        'profile': 'foot-walking',
        'range': [minutes * 60],
        'range_type': 'time'
    }

    isochrones = client.isochrones(**params)
    return isochrones, location

def find_green_areas_within_isochrone(isochrones_data, location, tags):
    # Erstellen eines GeoDataFrames aus dem Isochrone
    isochrone_gdf = gpd.GeoDataFrame.from_features(isochrones_data['features'], crs='EPSG:4326')
    
    # Laden der Grünflächen um den Punkt
    buffer_radius = 1000  # Standardradius in Metern
    gdf = ox.geometries_from_point(location, tags, dist=buffer_radius)
    gdf.crs = 'EPSG:4326'

    # Finden der Überschneidungen
    overlapping_areas = gpd.overlay(gdf, isochrone_gdf, how='intersection')
    return overlapping_areas

def load_and_display(address, minutes, api_key):
    try:
        isochrones_data, location = create_isochrone(address, minutes, api_key)
        tags = {'landuse': ['greenfield', 'grass', 'forest'], 'leisure': 'park'}

        overlapping_areas = find_green_areas_within_isochrone(isochrones_data, location, tags)

        m = folium.Map(location=[location[0], location[1]], zoom_start=14)
        folium.Marker([location[0], location[1]], popup='Ihre Adresse').add_to(m)

        for feature in isochrones_data['features']:
            folium.GeoJson(feature, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'fillOpacity': 0.5}).add_to(m)

        for _, area in overlapping_areas.iterrows():
            folium.GeoJson(area.geometry, style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'fillOpacity': 0.5}).add_to(m)

        display(m)
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

load_and_display(adresse, dauerSpaziergang, api_key)


C:\Users\j17jo\AppData\Local\Temp\ipykernel_28416\2219699130.py:31: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_point(location, tags, dist=buffer_radius)


## Nur Überschneidungen zwischen Isochrone und Grünflächen anzeigen
- gleiche Funktionen wie oben, aber abgeändert, dass nur grün angezeigt wird

In [8]:
import osmnx as ox
import geopandas as gpd
import folium
from IPython.display import display
import openrouteservice
from openrouteservice import client as ors_client
from shapely.geometry import shape
from shapely.ops import unary_union

def create_isochrone(address, minutes, api_key):
    client = ors_client.Client(key=api_key)
    location = ox.geocoder.geocode(address)
    coords = (location[1], location[0])

    params = {
        'locations': [coords],
        'profile': 'foot-walking',
        'range': [minutes * 60],
        'range_type': 'time'
    }

    isochrones = client.isochrones(**params)
    return isochrones, location

def find_green_areas_within_isochrone(isochrones_data, location, tags):
    # Erstellen eines GeoDataFrames aus dem Isochrone
    isochrone_gdf = gpd.GeoDataFrame.from_features(isochrones_data['features'], crs='EPSG:4326')
    
    # Laden der Grünflächen um den Punkt
    buffer_radius = 1000  # Standardradius in Metern
    gdf = ox.geometries_from_point(location, tags, dist=buffer_radius)
    gdf.crs = 'EPSG:4326'

    # Finden der Überschneidungen
    overlapping_areas = gpd.overlay(gdf, isochrone_gdf, how='intersection')
    return overlapping_areas

def load_and_display(address, minutes, api_key):
    try:
        isochrones_data, location = create_isochrone(address, minutes, api_key)
        tags = {'landuse': ['greenfield', 'grass', 'forest'], 'leisure': 'park'}

        overlapping_areas = find_green_areas_within_isochrone(isochrones_data, location, tags)

        m = folium.Map(location=[location[0], location[1]], zoom_start=14)
        folium.Marker([location[0], location[1]], popup='Ihre Adresse').add_to(m)

        # Nur die Überschneidungen anzeigen
        for _, area in overlapping_areas.iterrows():
            folium.GeoJson(
                area.geometry,
                style_function=lambda x: {'fillColor': 'green', 'color': 'green', 'fillOpacity': 0.7}
            ).add_to(m)

        display(m)
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

# adresse = "Zürich, Switzerland"
# dauerSpaziergang = 40  # Minuten

load_and_display(adresse, dauerSpaziergang, api_key)


C:\Users\j17jo\AppData\Local\Temp\ipykernel_28416\1030670400.py:31: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_point(location, tags, dist=buffer_radius)


In einem weiteren Schritt werden die Schnittflächen ermittelt. Hierbei wird eine Reoute ermittelt welche der Benutzer laufen könnte mit der Bedingung mindestens eine Gründfläche zu beinhalten. 

Hier wird die Route berechnet:

In [9]:
## Hier Berechnung Route einfügen

Hier werden alle Routen ausgegeben, die den Anforderungen entsprechen.

In [10]:
# Hier Code für Ausgabe Routen einfügen

## Anbindung PostGRESQL

In [11]:
import psycopg2


# Verbindungsdaten
host = "localhost"
dbname = "deine_datenbank"
user = "dein_user"
password = "12345678"

# Verbindung aufbauen
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# Einen Cursor erstellen
cur = conn.cursor()

# Eine SQL-Abfrage ausführen
cur.execute("SELECT * FROM deine_tabelle LIMIT 5;")

# Daten abrufen
rows = cur.fetchall()

for row in rows:
    print(row)

# Ressourcen freigeben
cur.close()
conn.close()


ModuleNotFoundError: No module named 'psycopg2'

## Benutzer UseCase

In diesem Kapitel wird ein Benutzer UseCase dargestellt

## UseCase

Bitte geben Sie Ihre Adresse ein:

Format:

Brandstrasse 1, Schlieren

In [ ]:
adresse = input("Bitte geben Sie die gewünschte Adresse ein:")
adresse = adresse + ", Switzerland"
print("Adresse:", adresse)

Adresse: hallo, Switzerland


Wie lange sollen ihre Spaziergänge mit Ihrem Viebeiner maximal dauern

In [ ]:
dauerSpaziergang = int(input("Wie lange sollen ihre Spaziergänge mit Ihrem Viebeiner maximal dauern"))
print("Dauer Spaziergang:", dauerSpaziergang, "Minuten")

Dauer Spaziergang: 15 Minuten


Wir ermitteln die beste Route für Sie....

In [ ]:
# Hier Funktionen aufrufen

Die beste Route für Sie:

In [ ]:
#Hier Karte mit Routen und Isochronen anzeigen

## Evaluation

Zusammenfassung der Ergebnisse